In [1]:
# =======================================================================
# CELDA 1: Instalación de todas las dependencias
# =======================================================================
print("Instalando bibliotecas necesarias...")
!pip install -q segment-anything-py
!pip install -q google-cloud-vision
!pip install -q opencv-python-headless matplotlib

# Descargar el modelo (checkpoint) de SAM
# Usaremos el modelo "vit_b" (base), que es el más rápido.
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

print("Instalación completa.")

Instalando bibliotecas necesarias...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 953.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.1/529.1 kB 11.5 MB/s eta 0:00:00
Instalación completa.


In [2]:
# =======================================================================
# CELDA 2: Autenticación
# =======================================================================
import os
from google.colab import files

print("Por favor, sube tu NUEVO archivo JSON de credenciales de Google Cloud.")

uploaded_files = files.upload()

# Obtener el nombre del archivo subido y configurar la variable de entorno
if not uploaded_files:
    print("\n¡No se subió ningún archivo! Ejecuta la celda de nuevo.")
else:
    json_key_filename = list(uploaded_files.keys())[0]
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_key_filename
    print(f"\nArchivo '{json_key_filename}' subido y autenticación configurada.")

Por favor, sube tu NUEVO archivo JSON de credenciales de Google Cloud.


Saving gen-lang-client-0027328417-65ea74948f12.json to gen-lang-client-0027328417-65ea74948f12.json

Archivo 'gen-lang-client-0027328417-65ea74948f12.json' subido y autenticación configurada.


In [3]:
# =======================================================================
# CELDA 3: Montar Google Drive
# =======================================================================
from google.colab import drive

print("Conectando con Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado en /content/drive")

Conectando con Google Drive...
Mounted at /content/drive
Google Drive montado en /content/drive


In [4]:
# =======================================================================
# CELDA 4: Definición de la ruta del archivo
# =======================================================================

map_image_path = "/content/drive/MyDrive/0.- Descargas/prueb1.png"


if os.path.exists(map_image_path):
    print(f"Archivo de mapa encontrado en: {map_image_path}")
else:
    print(f"¡ERROR! No se pudo encontrar el archivo en: {map_image_path}")
    print("Verifica que el nombre y la ruta en tu Drive sean correctos.")

Archivo de mapa encontrado en: /content/drive/MyDrive/0.- Descargas/prueb1.png


In [6]:
# =======================================================================
# CELDA 5: Carga de Modelos y Funciones Principales
# =======================================================================
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
from google.cloud import vision

# --- 1. Función de OCR (Cloud Vision) ---
# Esta función toma un recorte de imagen (en bytes) y devuelve el texto.
def get_text_from_roi(image_data):
    try:
        client = vision.ImageAnnotatorClient()
        image = vision.Image(content=image_data)

        # Usamos text_detection, que es bueno para texto corto y disperso
        response = client.text_detection(image=image)
        texts = response.text_annotations

        if texts:
            # texts[0] es la anotación completa, la limpiamos
            detected_text = texts[0].description.replace('\n', ' ').strip()
            return detected_text
        else:
            return None
    except Exception as e:
        print(f"Error en la API de Vision: {e}")
        return None

# --- 2. Carga del modelo SAM ---
print("Cargando modelo SAM (esto puede tardar un momento)...")
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth")
# Mover el modelo a la GPU de Colab si está disponible
sam.to(device="cuda" if torch.cuda.is_available() else "cpu")

# Configurar el generador automático de máscaras
# Ajustamos los parámetros para buscar "cosas" más grandes (polígonos)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=16,      # Menos puntos (más rápido)
    pred_iou_thresh=0.86,    # Umbral de calidad de la máscara
    min_mask_region_area=1000 # Ignorar polígonos muy pequeños
)
print("Modelo SAM cargado.")

Cargando modelo SAM (esto puede tardar un momento)...
Modelo SAM cargado.


In [9]:
# =======================================================================
# CELDA 6: Ejecución del Procesamiento (SAM + Vision)
# =======================================================================
import torch # Necesario para mover SAM a la GPU

print(f"Cargando imagen: {map_image_path}")
# Cargar la imagen con OpenCV
image_bgr = cv2.imread(map_image_path)
if image_bgr is None:
    print("Error al cargar la imagen. Verifica la ruta.")
else:
    # Convertir a RGB (que es lo que SAM espera)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    print("Iniciando segmentación con SAM...")
    # 1. SEGMENTACIÓN (SAM)
    # Esto genera una lista de todas las "máscaras" (polígonos)
    # Es la parte que más tarda
    masks = mask_generator.generate(image_rgb)
    print(f"SAM encontró {len(masks)} polígonos/segmentos.")

    resultados = [] # Aquí guardaremos {bbox, texto}

    # 2. OCR (Vision API) por cada máscara
    print("Iniciando reconocimiento de texto (OCR) en cada polígono...")

    for i, mask_data in enumerate(masks):
        # 'bbox' es [x_min, y_min, width, height]
        [x, y, w, h] = mask_data['bbox']

        # Recortar el polígono de la imagen original usando el Bounding Box
        # Esto es mucho más rápido que usar la máscara de polígono exacta
        recorte_rgb = image_rgb[y:y+h, x:x+w]

        # Convertir el recorte a formato de bytes (PNG) para la API
        _, buffer = cv2.imencode('.png', cv2.cvtColor(recorte_rgb, cv2.COLOR_RGB2BGR))
        image_bytes = buffer.tobytes()

        # 3. Llamar a la API de Vision con el recorte
        texto_detectado = get_text_from_roi(image_bytes)

        if texto_detectado:
            print(f"  Polígono {i}: BBox[{x},{y}] -> Texto: '{texto_detectado}'")
            # Guardamos el resultado
            resultados.append({
                "bbox": mask_data['bbox'],
                "text": texto_detectado
            })

    print("Procesamiento completado.")

Cargando imagen: /content/drive/MyDrive/0.- Descargas/prueb1.png
Iniciando segmentación con SAM...
SAM encontró 6 polígonos/segmentos.
Iniciando reconocimiento de texto (OCR) en cada polígono...
  Polígono 1: BBox[470,41] -> Texto: 'xx xx XXX xx'
  Polígono 2: BBox[0,563] -> Texto: '"KA'
  Polígono 3: BBox[2,39] -> Texto: 'G'
  Polígono 4: BBox[112,279] -> Texto: 'dillere de Agus Plutón de Mercabel Marabell'
Procesamiento completado.


In [10]:
# =======================================================================
# CELDA 7: Visualización de Resultados
# =======================================================================

print("Generando visualización final...")

if not 'resultados' in locals() or not resultados:
    print("No se encontraron resultados para visualizar.")
else:
    plt.figure(figsize=(20, 20))
    # Mostrar la imagen original
    plt.imshow(image_rgb)

    ax = plt.gca()
    ax.set_autoscale_on(False) # No cambiar la escala

    # Dibujar los Bounding Boxes y las etiquetas de texto
    for res in resultados:
        [x, y, w, h] = res['bbox']
        texto = res['text']

        # Dibujar el rectángulo (Bounding Box)
        # Usamos un color brillante (ej. rojo o cian)
        patch = plt.Rectangle((x, y), w, h, edgecolor='cyan', facecolor='none', lw=2)
        ax.add_patch(patch)

        # Dibujar el texto detectado
        # Lo ponemos justo encima del BBox
        plt.text(
            x, y - 5, # Posición (x, y-5 para que esté arriba)
            texto,
            color='yellow', # Color del texto
            fontsize=10,
            fontweight='bold',
            backgroundcolor='black' # Fondo negro para legibilidad
        )

    plt.title(f"Mapa Geológico con {len(resultados)} Etiquetas Detectadas")
    plt.axis('off') # Ocultar ejes
    plt.show()

Output hidden; open in https://colab.research.google.com to view.